In [6]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


# Import BeutifulSoup and requests library

In [26]:
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
import folium # map rendering library

# Get wikipedia page in lxml format

In [2]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')


# In below cell we will get the values of the table into a list form and then add that list to csv

In [3]:
import csv
output_data=[]
final=[]
csv_file=open('PostalCode.csv','w')
csv_writer=csv.writer(csv_file)

for tb in soup.find_all('tr'):
    data=tb.text
    if  "Canadian postal codes" in data:
        break
    else:
        output_data.append(data)

for i in output_data:
    split_value=i.split('\n')
    split_value.remove("")
    final.append(split_value)
    
for data in final:
    csv_writer.writerow(data)
csv_file.close()

# Getting csv into data frame

In [4]:
import pandas as pd
df=pd.read_csv('PostalCode.csv')
df=df[['Postal Code','Borough','Neighbourhood']]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


# Remove rows that consist Not assigned value for Borough column

In [5]:
df2=df[df.Borough!="Not assigned"]
df2.head(5)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Reset indexing

In [6]:
df2.reset_index(inplace = True, drop = True) 
df2

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Shape Attribute

In [7]:
df2.shape

(103, 3)

# Get Cordinate data frame

In [8]:
cord_df=pd.read_csv('Geospatial_Coordinates.csv')
cord_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


# Merging cordinates data frame with Postal Code data frame

In [9]:
df3=pd.merge(df2,cord_df,on='Postal Code')
df3

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
